In [1]:
import pandas as pd
import glob
import re

In [2]:
df = pd.read_csv("/home/smitesh22/Data/extent.csv")

In [3]:
filenames_DEM = set(df.filename)

In [4]:
filenames_GIS = glob.glob("/home/smitesh22/Data/GIS/*.zip")

In [5]:
filenames_GIS = {re.search(r'/([^/]+)\.zip$', name).group(1) for name in filenames_GIS}

In [6]:
assert filenames_GIS == filenames_DEM

In [15]:
from osgeo import ogr, osr

In [16]:
folders = glob.glob("/home/smitesh22/Data/GIS Extracted/")


In [17]:
folders

['/home/smitesh22/Data/GIS Extracted/']

In [9]:
# Open the shapefile with GDAL
driver = ogr.GetDriverByName('ESRI Shapefile')
dataset = driver.Open("/home/smitesh22/jordan-latest-free.shp", 0)
layer = dataset.GetLayer()

In [10]:
extent = layer.GetExtent()

In [11]:
min_x, max_x, min_y, max_y = extent

# Print the extent
print("Min X:", min_x)
print("Max X:", max_x)
print("Min Y:", min_y)
print("Max Y:", max_y)

Min X: 34.966434
Max X: 38.2006278
Min Y: 29.1922604
Max Y: 32.729977


In [12]:
driver = ogr.GetDriverByName('ESRI Shapefile')
dataset = driver.Open("/home/smitesh22/Data/GIS/N29E075_wgs84/planet_75,29_76,30/shape", 0)
layer = dataset.GetLayer()

In [13]:
extent = layer.GetExtent()

In [14]:
min_x, max_x, min_y, max_y = extent

# Print the extent
print("Min X:", min_x)
print("Max X:", max_x)
print("Min Y:", min_y)
print("Max Y:", max_y)

Min X: 75.0067882
Max X: 75.9998575
Min Y: 29.0063107
Max Y: 29.9982034
